# Frictionless Product Categorization

- **Author:** Jose Vicente Ruiz Cepeda (jr3660)
- **Course:** COMS W 4995 - Deep Learning for Computer Vision
- **Assignment**: Final project

## Context

Explain some of the context of the problem here.

## Code

### Imports

In [1]:
import keras # Keras 1.2.2 assumed.
from keras import optimizers

from keras.models import Model

from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception
from keras.applications import VGG16
from keras.applications import VGG19

from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.inception_v3 import preprocess_input

from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten

from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

import os
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import confusion_matrix

# Required to avoid errors with the images.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline

Using TensorFlow backend.


### Constants

In [2]:
CATEGORIES = {
    0: 'photo',
    1: 'electronics',
    2: 'events',
    3: 'instruments',
    4: 'tools',
    5: 'sports',
    6: 'caravans',
    7: 'others'
}

SUBCATEGORIES = {
    0:  '360',
    1:  'action_cameras',
    2:  'drones',
    3:  'instant_cameras',
    4:  'kits',
    5:  'lenses',
    6:  'lighting',
    7:  'others',
    8:  'photo_cameras',
    9:  'projectors',
    10: 'sound',
    11: 'supports',
    12: 'video_cameras'
}

### Functions

In [3]:
# Modified from http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=60)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "%d%%" % int(cm[i, j]*100),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [4]:
def show_sample(X, y, class_names, prediction=-1):
    im = X
    print y
    #y = np.flip(y, axis=0)
    y_label = class_names[y]
    plt.imshow(im)
    if prediction >= 0:
        plt.title("Class = %s, Predict = %s" % (y_label, class_names[prediction]))
    else:
        plt.title("Class = %s" % (y_label))

    plt.axis('on')
    plt.show()

### Part I - Categories

#### Training

In [5]:
train_data_dir = './data/train'
validation_data_dir = './data/validation'
test_data_dir = './data/test'

In [6]:
class Categorizer:
    def __init__(self, classes):
        self.classes = classes
        
    def build_model(self, 
                    ImagenetModel=ResNet50,
                    weights='imagenet',
                    num_dense_layer_units=256,
                    dropout=0.5,
                    multiclass_output=False,
                    verbose=False):
        # Keras models are functions, not classes, so we have to check which one is it like this.
        if ImagenetModel.func_name == 'InceptionV3' or ImagenetModel.func_name == 'Xception':
            self.img_width, self.img_height = 299, 299
        else:
            self.img_width, self.img_height = 224, 224

        # First, let's load the model with ImageNet weights and without the top layer.
        # This will take some time the first time, since the weights have to be downloaded.
        model_weights = weights if weights == 'imagenet' else None
        model = ImagenetModel(
            weights=model_weights,
            include_top=False,
            input_shape=(self.img_width, self.img_height, 3)
        )
        self.bottom_model = model

        # Now, let's create the top layers adapted to our problem. 
        preds = self._build_top_model(
            num_dense_layer_units=num_dense_layer_units,
            dropout=dropout,
            multiclass_output=multiclass_output)

        # Combine both models to get the final one.
        self.model = Model(model.input, preds)

        if verbose:
            self.model.summary()
            
        # If weights is a string different from ImageNet, path to weights is assumed.
        if os.path.exists(weights):
            self.model.load_weights(weights)

        return self
    
    def change_top_model(self,
                         new_classes,
                         num_dense_layer_units=256,
                         dropout=0.5,
                         multiclass_output=False,
                         verbose=False):
        self.classes = new_classes
        model = self.bottom_model
        preds = self._build_top_model(
            num_dense_layer_units=num_dense_layer_units,
            dropout=dropout,
            multiclass_output=multiclass_output)

        self.model = Model(model.input, preds)
        if verbose:
            self.model.summary()

        return self
        
    def _build_top_model(self,
                         num_dense_layer_units=256,
                         dropout=0.5,
                         multiclass_output=False):
        # We use the Functional API of Keras. 
        # https://keras.io/getting-started/functional-api-guide/
        model = self.bottom_model
        x = model.output
        x = Flatten(input_shape=model.output_shape[1:])(x)
        x = Dense(num_dense_layer_units, activation='relu')(x)
        x = Dropout(dropout)(x)
        preds = Dense(
            len(self.classes),
            activation='sigmoid' if multiclass_output else 'softmax')(x)
        self.num_top_layers = 4
        
        return preds
    
    def compile_model(self, optimizer=optimizers.Adagrad(lr=0.001)):
        self.is_compiled = True
        self.optimizer = optimizer
        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy']
        )
            
        return self
    
    def _recompile_model(self):
        if self.is_compiled != True:
            raise Error("Model has to be compiled first.")

        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=self.optimizer,
            metrics=['accuracy']
        )
    
    def fine_tune(self,
                  train_data_dir,
                  validation_data_dir,
                  batch_size=16,
                  num_only_top_epochs=10,
                  num_whole_model_epochs=40,
                  best_model_path=None,
                  tensorboard_logs_path=None,
                  reduce_learning_rate=True):
        
        # Augmentation configurations for training and validation.
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
        validation_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')

        validation_generator = validation_datagen.flow_from_directory(
            validation_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')
        
        if num_only_top_epochs > 0:
            # Freeze everything except the top layers, before training.
            for layer in self.model.layers[:-self.num_top_layers]:
                layer.trainable = False 

            print "Starting with top layers training..."
            self._fit_generator(
                train_generator,
                validation_generator,
                batch_size,
                num_only_top_epochs,
                best_model_path=best_model_path,
                tensorboard_logs_path=tensorboard_logs_path,
                tensorboard_logs_path_suffix='Top',
                reduce_learning_rate=True)
            print "Top layers training done."
            
        if num_whole_model_epochs > 0:
            # Unfreeze everything and train for some more epochs.
            for layer in self.model.layers:
                layer.trainable = True
            
            print "Starting with whole model training..."
            self._fit_generator(
                train_generator,
                validation_generator,
                batch_size,
                num_whole_model_epochs,
                best_model_path=best_model_path,
                tensorboard_logs_path=tensorboard_logs_path,
                tensorboard_logs_path_suffix='Whole',
                reduce_learning_rate=True)
            print "Whole model training done"
        
        return self
    
    def predict(self, X_batch):
        return self.model.predict(X_batch)
        
    def predict_generator(self, test_data_dir):
        # As explained in https://github.com/fchollet/keras/issues/3896,
        # there is an unresolved issue with predict_generator that causes
        # errors when the number of samples is not divisible by the batch.
        # size. To avoid this problem, let's use batch_size=1, for now.
        batch_size = 1
        
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            test_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')
        
        
        return self.model.predict_generator(
            test_generator,
            test_generator.n//batch_size)
    
    def predict_classes(self, test_data_dir):
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            test_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')
        
        for X_batch, Y_batch in train_generator:
            for i in range(len(Y_batch)):
                show_sample(X_batch[i, :, :, :], Y_batch[i])
            break
        
    
    def evaluate(self, test_data_dir, batch_size=32):
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            test_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=self.classes.values(),
            class_mode='categorical')
        
        return self.model.evaluate_generator(
            test_generator,
            test_generator.n//batch_size)
    
    # Private methods.
    def _fit_generator(self,
                       train_generator,
                       validation_generator,
                       batch_size,
                       num_epochs,
                       best_model_path=None,
                       tensorboard_logs_path=None,
                       tensorboard_logs_path_suffix='',
                       reduce_learning_rate=True):
        # Recompile model before training to increase efficiency in
        # case of frozen layers.
        self._recompile_model()
        
        # Save the best model based on validation accuracy.
        if best_model_path:
            model_checkpoint = ModelCheckpoint(
                best_model_path,
                monitor='val_acc',
                save_best_only=True,
                save_weights_only=False,
                period=1
            )
        
        # Log epochs information like loss and accuracy to review it
        # afterwards using TensorBoard.
        if tensorboard_logs_path:
            if tensorboard_logs_path[-1] == '/':
                tensorboard_logs_path = tensorboard_logs_path[:-1]
            tensorboard = TensorBoard(
                log_dir=tensorboard_logs_path+'-'+tensorboard_logs_path_suffix+'/')
        
        # Reduce learning rate i
        if reduce_learning_rate:
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
        
        self.model.fit_generator(
            train_generator,
            samples_per_epoch=train_generator.n//batch_size,
            nb_epoch=num_epochs,
            validation_data=validation_generator,
            nb_val_samples=validation_generator.n//batch_size,
            callbacks=[model_checkpoint, tensorboard, reduce_lr])

#### Testing

## Part II: Subcategories

In [7]:
subcategories_train_data_dir = './subcategories-data/train'
subcategories_validation_data_dir = './subcategories-data/validation'
subcategories_test_data_dir = './subcategories-data/test'

In [8]:
Categorizer(CATEGORIES). \
    build_model(VGG16, weights='best_models/VGG16.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/VGG16-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG16-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/20
 32/383 [=>............................] - ETA: 32s - loss: 8.6754 - acc: 0.0000e+00

/usr/lib/python2.7/dist-packages/PIL/Image.py:857: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


368/383 [===========================>..] - ETA: 0s - loss: 7.2538 - acc: 0.0299

/usr/local/lib/python2.7/dist-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


384/383 [==============================] - 12s - loss: 7.2487 - acc: 0.0339 - val_loss: 7.6780 - val_acc: 0.0208
Epoch 2/20
384/383 [==============================] - 10s - loss: 7.4044 - acc: 0.0573 - val_loss: 9.2126 - val_acc: 0.1562
Epoch 3/20
384/383 [==============================] - 9s - loss: 6.5500 - acc: 0.0781 - val_loss: 5.2241 - val_acc: 0.0208
Epoch 4/20
384/383 [==============================] - 9s - loss: 6.4197 - acc: 0.0677 - val_loss: 5.0749 - val_acc: 0.0312
Epoch 5/20
384/383 [==============================] - 9s - loss: 6.5587 - acc: 0.0599 - val_loss: 6.8991 - val_acc: 0.0208
Epoch 6/20
384/383 [==============================] - 9s - loss: 6.9124 - acc: 0.0807 - val_loss: 4.9256 - val_acc: 0.0521
Epoch 7/20
384/383 [==============================] - 9s - loss: 6.2123 - acc: 0.0547 - val_loss: 6.1195 - val_acc: 0.0208
Epoch 8/20
384/383 [==============================] - 9s - loss: 6.2813 - acc: 0.0573 - val_loss: 4.6272 - val_acc: 0.0312
Epoch 9/20
384/383 [=====

<__main__.Categorizer instance at 0x7fd94c6f9e18>

In [9]:
Categorizer(SUBCATEGORIES). \
    build_model(VGG16, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/VGG16-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG16-subcats-imagenet'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/20
384/383 [==============================] - 10s - loss: 2.4222 - acc: 0.1484 - val_loss: 2.3078 - val_acc: 0.1875
Epoch 2/20
384/383 [==============================] - 9s - loss: 2.2832 - acc: 0.2135 - val_loss: 2.1766 - val_acc: 0.1667
Epoch 3/20
384/383 [==============================] - 10s - loss: 2.2636 - acc: 0.1771 - val_loss: 2.2526 - val_acc: 0.2188
Epoch 4/20
384/383 [==============================] - 9s - loss: 2.2105 - acc: 0.2240 - val_loss: 2.2415 - val_acc: 0.2188
Epoch 5/20
384/383 [==============================] - 10s - loss: 2.2281 - acc: 0.1953 - val_loss: 2.1850 - val_acc: 0.2604
Epoch 6/20
384/383 [==============================] - 10s - loss: 2.2218 - acc: 0.2292 - val_loss: 2.1144 - val_acc: 0.3125
Epoch 7/20
384/383 [==============================] - 9s - loss: 2.2144 - acc: 0.2708 - val_loss: 2.2574 - val_acc: 0.1562
Epoch 8/20
3

<__main__.Categorizer instance at 0x7fd94c61d1b8>

In [10]:
Categorizer(CATEGORIES). \
    build_model(VGG19, weights='best_models/VGG19.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/VGG19-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG19-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/20
384/383 [==============================] - 12s - loss: 8.3934 - acc: 0.0703 - val_loss: 6.5601 - val_acc: 0.1250
Epoch 2/20
384/383 [==============================] - 11s - loss: 7.0048 - acc: 0.0885 - val_loss: 5.1828 - val_acc: 0.0625
Epoch 3/20
384/383 [==============================] - 11s - loss: 5.7770 - acc: 0.0885 - val_loss: 5.4890 - val_acc: 0.0521
Epoch 4/20
384/383 [==============================] - 11s - loss: 6.3701 - acc: 0.0599 - val_loss: 5.0461 - val_acc: 0.0625
Epoch 5/20
384/383 [==============================] - 11s - loss: 5.6336 - acc: 0.0495 - val_loss: 5.4890 - val_acc: 0.0208
Epoch 6/20
384/383 [==============================] - 11s - loss: 5.8843 - acc: 0.0573 - val_loss: 6.2271 - val_acc: 0.0208
Epoch 7/20
384/383 [==============================] - 11s - loss: 5.5374 - acc: 0.0443 - val_loss: 4.4410 - val_acc: 0.0521
Epoch 8/2

<__main__.Categorizer instance at 0x7fd8eeb980e0>

In [11]:
Categorizer(CATEGORIES). \
    build_model(VGG19, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/VGG19-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG19-subcats-imagenet'
    )

Found 756 images belonging to 8 classes.
Found 189 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/20
48/47 [==============================] - 2s - loss: 1.8944 - acc: 0.4583 - val_loss: 1.3171 - val_acc: 1.0000
Epoch 2/20
48/47 [==============================] - 1s - loss: 1.1990 - acc: 0.9792 - val_loss: 0.8400 - val_acc: 1.0000
Epoch 3/20
48/47 [==============================] - 1s - loss: 0.8526 - acc: 1.0000 - val_loss: 0.4948 - val_acc: 1.0000
Epoch 4/20
48/47 [==============================] - 1s - loss: 0.5782 - acc: 1.0000 - val_loss: 0.3033 - val_acc: 1.0000
Epoch 5/20
48/47 [==============================] - 1s - loss: 0.4279 - acc: 1.0000 - val_loss: 0.2027 - val_acc: 1.0000
Epoch 6/20
48/47 [==============================] - 1s - loss: 0.2969 - acc: 1.0000 - val_loss: 0.1616 - val_acc: 1.0000
Epoch 7/20
48/47 [==============================] - 1s - loss: 0.3062 - acc: 1.0000 - val_loss: 0.1141 - val_acc: 1.0000
Epoch 8/20
48/47 [================

<__main__.Categorizer instance at 0x7fd8efff7ef0>

In [12]:
Categorizer(CATEGORIES). \
    build_model(ResNet50, weights='best_models/ResNet50.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/ResNet50-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/ResNet50-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/20
384/383 [==============================] - 14s - loss: 2.5902 - acc: 0.1016 - val_loss: 2.4922 - val_acc: 0.0938
Epoch 2/20
384/383 [==============================] - 7s - loss: 2.4882 - acc: 0.1276 - val_loss: 2.3792 - val_acc: 0.2083
Epoch 3/20
384/383 [==============================] - 6s - loss: 2.4443 - acc: 0.1589 - val_loss: 2.4097 - val_acc: 0.2083
Epoch 4/20
384/383 [==============================] - 7s - loss: 2.4470 - acc: 0.1589 - val_loss: 2.3821 - val_acc: 0.2188
Epoch 5/20
384/383 [==============================] - 7s - loss: 2.4439 - acc: 0.1693 - val_loss: 2.3257 - val_acc: 0.2292
Epoch 6/20
384/383 [==============================] - 6s - loss: 2.4057 - acc: 0.1875 - val_loss: 2.3081 - val_acc: 0.1875
Epoch 7/20
384/383 [==============================] - 7s - loss: 2.3499 - acc: 0.2135 - val_loss: 2.2927 - val_acc: 0.2812
Epoch 8/20
384/

<__main__.Categorizer instance at 0x7fd889f167e8>

In [13]:
Categorizer(CATEGORIES). \
    build_model(ResNet50, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/ResNet50-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/ResNet50-subcats-imagenet'
    )

Found 756 images belonging to 8 classes.
Found 189 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/20
48/47 [==============================] - 20s - loss: 1.9791 - acc: 0.2083 - val_loss: 1.9496 - val_acc: 0.0000e+00
Epoch 2/20
48/47 [==============================] - 1s - loss: 1.7023 - acc: 0.5625 - val_loss: 1.8888 - val_acc: 0.0000e+00
Epoch 3/20
48/47 [==============================] - 1s - loss: 1.5914 - acc: 0.7292 - val_loss: 1.8558 - val_acc: 0.0000e+00
Epoch 4/20
48/47 [==============================] - 1s - loss: 1.4980 - acc: 0.7708 - val_loss: 1.7965 - val_acc: 0.1250
Epoch 5/20
48/47 [==============================] - 1s - loss: 1.4266 - acc: 0.8125 - val_loss: 1.7914 - val_acc: 0.0625
Epoch 6/20
48/47 [==============================] - 1s - loss: 1.3916 - acc: 0.8958 - val_loss: 1.7371 - val_acc: 0.5000
Epoch 7/20
48/47 [==============================] - 1s - loss: 1.2867 - acc: 0.9583 - val_loss: 1.7444 - val_acc: 0.4375
Epoch 8/20
48/47 [===

<__main__.Categorizer instance at 0x7fd8eeb98680>

In [14]:
Categorizer(CATEGORIES). \
    build_model(Xception, weights='best_models/Xception.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/Xception-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/Xception-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/20
384/383 [==============================] - 59s - loss: 2.3726 - acc: 0.2448 - val_loss: 2.2693 - val_acc: 0.2812
Epoch 2/20
384/383 [==============================] - 15s - loss: 2.2533 - acc: 0.2188 - val_loss: 2.1655 - val_acc: 0.2604
Epoch 3/20
384/383 [==============================] - 15s - loss: 2.1450 - acc: 0.2760 - val_loss: 2.1360 - val_acc: 0.1667
Epoch 4/20
384/383 [==============================] - 19s - loss: 2.1481 - acc: 0.2526 - val_loss: 2.0675 - val_acc: 0.3021
Epoch 5/20
384/383 [==============================] - 15s - loss: 2.1331 - acc: 0.2500 - val_loss: 2.0328 - val_acc: 0.2292
Epoch 6/20
384/383 [==============================] - 15s - loss: 2.1079 - acc: 0.2760 - val_loss: 1.9534 - val_acc: 0.2396
Epoch 7/20
384/383 [==============================] - 15s - loss: 2.0844 - acc: 0.2630 - val_loss: 2.1302 - val_acc: 0.2500
Epoch 8/2

<__main__.Categorizer instance at 0x7fd8a0138248>

In [15]:
Categorizer(CATEGORIES). \
    build_model(Xception, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/Xception-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/Xception-subcats-imagenet'
    )

Found 756 images belonging to 8 classes.
Found 189 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/20
48/47 [==============================] - 33s - loss: 1.4297 - acc: 0.7083 - val_loss: 0.1380 - val_acc: 1.0000
Epoch 2/20
48/47 [==============================] - 2s - loss: 0.2470 - acc: 1.0000 - val_loss: 0.0190 - val_acc: 1.0000
Epoch 3/20
48/47 [==============================] - 2s - loss: 0.0895 - acc: 1.0000 - val_loss: 0.0115 - val_acc: 1.0000
Epoch 4/20
48/47 [==============================] - 2s - loss: 0.0558 - acc: 1.0000 - val_loss: 0.0114 - val_acc: 1.0000
Epoch 5/20
48/47 [==============================] - 2s - loss: 0.0306 - acc: 1.0000 - val_loss: 0.0048 - val_acc: 1.0000
Epoch 6/20
48/47 [==============================] - 2s - loss: 0.0254 - acc: 1.0000 - val_loss: 0.0039 - val_acc: 1.0000
Epoch 7/20
48/47 [==============================] - 2s - loss: 0.0335 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 1.0000
Epoch 8/20
48/47 [===============

<__main__.Categorizer instance at 0x7fd889ef03b0>

In [16]:
Categorizer(CATEGORIES). \
    build_model(InceptionV3, weights='best_models/InceptionV3.hdf5'). \
    change_top_model(SUBCATEGORIES, multiclass_output=True).compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/InceptionV3-subcats.hdf5',
        tensorboard_logs_path='tensorboard_logs/InceptionV3-subcats'
    )

Found 6133 images belonging to 13 classes.
Found 1541 images belonging to 13 classes.
Starting with top layers training...
Epoch 1/20
384/383 [==============================] - 97s - loss: 2.5102 - acc: 0.1719 - val_loss: 2.2573 - val_acc: 0.1875
Epoch 2/20
384/383 [==============================] - 10s - loss: 2.3972 - acc: 0.1432 - val_loss: 2.3492 - val_acc: 0.1667
Epoch 3/20
384/383 [==============================] - 14s - loss: 2.2927 - acc: 0.1667 - val_loss: 2.2755 - val_acc: 0.2292
Epoch 4/20
384/383 [==============================] - 10s - loss: 2.3820 - acc: 0.1823 - val_loss: 2.2676 - val_acc: 0.1875
Epoch 5/20
384/383 [==============================] - 10s - loss: 2.3424 - acc: 0.1901 - val_loss: 2.2615 - val_acc: 0.1562
Epoch 6/20
384/383 [==============================] - 10s - loss: 2.3408 - acc: 0.2135 - val_loss: 2.5382 - val_acc: 0.1562
Epoch 7/20
384/383 [==============================] - 10s - loss: 2.3299 - acc: 0.1667 - val_loss: 2.2632 - val_acc: 0.1771
Epoch 8/2

<__main__.Categorizer instance at 0x7fd8207b8cb0>

In [17]:
Categorizer(CATEGORIES). \
    build_model(InceptionV3, weights='imagenet', multiclass_output=True). \
    compile_model(). \
    fine_tune(
        subcategories_train_data_dir,
        subcategories_validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=50,
        best_model_path='best_models/InceptionV3-subcats-imagenet.hdf5',
        tensorboard_logs_path='tensorboard_logs/InceptionV3-subcats-imagenet'
    )

Found 756 images belonging to 8 classes.
Found 189 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/20
48/47 [==============================] - 135s - loss: 2.0260 - acc: 0.5000 - val_loss: 0.6271 - val_acc: 0.9375
Epoch 2/20
48/47 [==============================] - 1s - loss: 0.7149 - acc: 0.8958 - val_loss: 0.4295 - val_acc: 0.8750
Epoch 3/20
48/47 [==============================] - 5s - loss: 0.4220 - acc: 0.9167 - val_loss: 0.0852 - val_acc: 1.0000
Epoch 4/20
48/47 [==============================] - 1s - loss: 0.4788 - acc: 0.9792 - val_loss: 0.1760 - val_acc: 0.9375
Epoch 5/20
48/47 [==============================] - 1s - loss: 0.4308 - acc: 0.9375 - val_loss: 0.0357 - val_acc: 1.0000
Epoch 6/20
48/47 [==============================] - 1s - loss: 0.3648 - acc: 0.9583 - val_loss: 0.0137 - val_acc: 1.0000
Epoch 7/20
48/47 [==============================] - 1s - loss: 0.3622 - acc: 0.9375 - val_loss: 0.0386 - val_acc: 1.0000
Epoch 8/20
48/47 [==============

<__main__.Categorizer instance at 0x7fd811bdd878>

# Sandbox